IDEEN / TODOS:<br>

- allgemeines zu sentiment analysis finden (literatur)
- Medium Artikel zum Dataset: https://towardsdatascience.com/sentiment-analysis-and-product-recommendation-on-amazons-electronics-dataset-reviews-part-1-6b340de660c2

In [1]:
import pandas as pd

<h1>Inhaltsverzeichnis<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Einleitung" data-toc-modified-id="Einleitung-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Einleitung</a></span></li><li><span><a href="#Das-Korpus" data-toc-modified-id="Das-Korpus-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Das Korpus</a></span></li><li><span><a href="#Die-verschiedenen-Word-Embeddings" data-toc-modified-id="Die-verschiedenen-Word-Embeddings-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Die verschiedenen Word Embeddings</a></span></li><li><span><a href="#Experimente" data-toc-modified-id="Experimente-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Experimente</a></span></li><li><span><a href="#Schlussbetrachtung" data-toc-modified-id="Schlussbetrachtung-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Schlussbetrachtung</a></span></li><li><span><a href="#Literaturverzeichnis" data-toc-modified-id="Literaturverzeichnis-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Literaturverzeichnis</a></span></li></ul></div>

# Einleitung

TODO: Allgemein über Sentiment Analysis


In dieser Arbeit wird eine Sentiment Analysis von Nutzerreviews des Onlineversandhändlers **Amazon** mithilfe eines Convolutional Neural Networks (CNN) durchgeführt. Es soll untersucht werden, welche Word Embeddings in Kombination mit welchen Parametern des Neuronalen Netzes die besten Ergebnisse liefern. Als Word Embeddings werden **GloVe**, **FastText** und **BERT** Embeddings miteinander verglichen. Die Ergebnisse der Sentiment Analysis in Kombination mit den Word Embeddings soll mit einem weiteren Experiment verglichen werden, beim dem das Korpus mithilfe des BERT-Modells *fine-tuned* wird


Diese Ergebnisse sollen mit einem 



Besonderheiten beim Nutzerreview-Korpus sind die Kürze der Texte und die fehlerhafte Orthographie.



TODO: warum cnns?
- hier wird funktion für text erklärt: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb
- best practices machine learning mastery: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb
- gutes netzwerk: https://arxiv.org/pdf/2004.03705.pdf (S. 27)
- spezielles CNN, **KimCNN**: https://towardsdatascience.com/identifying-hate-speech-with-bert-and-cnn-b7aa2cddd60d

TODO: fragstellung
- Word Embeddings (GloVe, FastText) mit CNN vs. BERT/XLNet Transformer, die die Embeddings nutzen (SOTA)
- Bert kann nicht so einfach mit CNN verwendet werden, da Modell mitgeliefert werden muss
- interessant, wie CNN gegen BERT, welches eine Art Weiterentwicklung von RNN ist, funktioniert
- verschiedene CNN Netze?


# Das Korpus

Das verwendete Korpus ist ein Sammlung von englischsprachigen Nutzerreviews zu den Produkten des Onlineversandhändlers **Amazon** von Julian McAuley ([Quelle](https://nijianmo.github.io/amazon/index.html)). Der Zeitraum der Veröffentlichungsdaten der Reviews im originalen Korpus liegt zwischen dem Mai 1996 und dem Oktober 2018. Diese Zeitspanne umfasst ~ 233 Millionen Reviews aus 29 verschiedenen Produktkategorien. Zu jedem Produkt stehen die Bewertung in einer Skala von 1 bis 5 (sehr schlecht bis sehr gut) zur Verfügung, der Reviewtext, die Anzahl der "Nützlich"-Votierungen, eine Verifizierung von Amazon, die Produkt-Metadaten und weitere Links. 

Für diese Arbeit wurde eine verkürzte Version des Korpus verwendet. Alle Produktreviews stammen aus der Kategorie "Elektronik" und lediglich aus dem Jahre 2018. Es wurden nur Reviews berücksichtigt, die zu jeder ausgewählten Metainformation ("Bewertung", "Nutzername", "Reviewtext", "Verifizierung", "Datum") Werte enthielten. Das resultiernde Korpus zeigte hinsichtlich der Klassenverteilung eine starke Unausgeglichenheit, weshalb mithilfe von zufälligem Downsampling zu jeder Klasse 15000 Nutzerreviews ausgewählt wurden, um ein ausgeglichenes Korpus zu erhalten (Größe: 75000 Reviews). Ein Einblick in das Korpus wird in der nächsten Zeile gegeben.

In [3]:
corpus = pd.read_csv("../corpora/small_amazon_reviews_electronic.csv")
corpus.head(3)

,rating,name,review,verified,vote,date
0,1.0,Mike L,Bought for Christmas present for my Grandson h...,True,0.0,01.02.2018
1,1.0,Gustavo Villalta Woltke,Broken in months,True,0.0,23.05.2018
2,1.0,David,The latest driver for this product on the Asus...,False,0.0,15.05.2018


# Die verschiedenen Word Embeddings

TODO: kurze Zusammenfassung der verwendeten Embeedings

In [16]:
import json

In [30]:
def load_jsonl_to_df(path):
    """ Create dataframe from a JSON lines file. """
    data = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    df = pd.DataFrame(data)
    if type(df["review"][0]) == list:
        df["review"] = df.review.apply(" ".join)
    return df

In [31]:
load_jsonl_to_df('../corpora/splits/train.json')

,rating,name,review,verified,vote,date
0,5.0,Jerry Keisler,I use it every day on my laptop for about hour...,True,2.0,01.02.2018
1,2.0,Amazo,Updated rating for drive failing after one mon...,True,0.0,26.02.2018
2,2.0,Amazon Customer,Very hard to shape the stand to a certain posi...,True,0.0,03.01.2018
3,1.0,David L Clark,This is very cheaply made I can not get my I p...,True,0.0,26.01.2018
4,1.0,Amazon Customer,It is works sometimes,True,0.0,06.04.2018
...,...,...,...,...,...,...
44995,3.0,ZACK M SHAHIN,We are still trying to get this splitter to wo...,True,0.0,04.08.2018
44996,5.0,Mary King,Worked great,True,0.0,10.04.2018
44997,4.0,JNF,Work well when it wants to It keeps disconnect...,True,0.0,07.04.2018
44998,5.0,Amazon Customer,Just what I needed for my Canon M lens,True,0.0,11.06.2018


In [11]:
webpage_data = load_jsonl('../corpora/splits/train.json')
db_data = []
db_cols = ['rating', "name", "review", "verified", "vote", "date"]
for d in webpage_data:
    db_data.append([])
    for col in db_cols:
        db_data[-1].append(d.get(col, float('nan')))
        
df = pd.DataFrame(db_data, columns=db_cols)

Loaded 45000 records from ../corpora/splits/train.json


In [14]:
pd.DataFrame(webpage_data)

,rating,name,review,verified,vote,date
0,5.0,Jerry Keisler,"[I, use, it, every, day, on, my, laptop, for, ...",True,2.0,01.02.2018
1,2.0,Amazo,"[Updated, rating, for, drive, failing, after, ...",True,0.0,26.02.2018
2,2.0,Amazon Customer,"[Very, hard, to, shape, the, stand, to, a, cer...",True,0.0,03.01.2018
3,1.0,David L Clark,"[This, is, very, cheaply, made, I, can, not, g...",True,0.0,26.01.2018
4,1.0,Amazon Customer,"[It, is, works, sometimes]",True,0.0,06.04.2018
...,...,...,...,...,...,...
44995,3.0,ZACK M SHAHIN,"[We, are, still, trying, to, get, this, splitt...",True,0.0,04.08.2018
44996,5.0,Mary King,"[Worked, great]",True,0.0,10.04.2018
44997,4.0,JNF,"[Work, well, when, it, wants, to, It, keeps, d...",True,0.0,07.04.2018
44998,5.0,Amazon Customer,"[Just, what, I, needed, for, my, Canon, M, lens]",True,0.0,11.06.2018


In [12]:
df

,rating,name,review,verified,vote,date
0,5.0,Jerry Keisler,"[I, use, it, every, day, on, my, laptop, for, ...",True,2.0,01.02.2018
1,2.0,Amazo,"[Updated, rating, for, drive, failing, after, ...",True,0.0,26.02.2018
2,2.0,Amazon Customer,"[Very, hard, to, shape, the, stand, to, a, cer...",True,0.0,03.01.2018
3,1.0,David L Clark,"[This, is, very, cheaply, made, I, can, not, g...",True,0.0,26.01.2018
4,1.0,Amazon Customer,"[It, is, works, sometimes]",True,0.0,06.04.2018
...,...,...,...,...,...,...
44995,3.0,ZACK M SHAHIN,"[We, are, still, trying, to, get, this, splitt...",True,0.0,04.08.2018
44996,5.0,Mary King,"[Worked, great]",True,0.0,10.04.2018
44997,4.0,JNF,"[Work, well, when, it, wants, to, It, keeps, d...",True,0.0,07.04.2018
44998,5.0,Amazon Customer,"[Just, what, I, needed, for, my, Canon, M, lens]",True,0.0,11.06.2018


In [5]:
pd.read_json("../corpora/splits/train.json", orient="index")

ValueError: Trailing data

# Experimente

TODO<br>:


## Aufbau

TODO
- Stoppwörter wurden beibehalten, da ansonsten zu wenig Text

# Schlussbetrachtung

# Literaturverzeichnis